# Import

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import numpy as np
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.models import Model
from keras.models import Sequential

# Opening datasets

In [ ]:
if False:
    df2 = pd.read_excel('Data/data.xlsx', sheet_name='Mesures', header=1)
    df2.drop(index=0, inplace=True)

    df1 = pd.read_excel('Data/Raw_data_from26.10to09.11.2022.xlsx', sheet_name='Mesures', header=1)
    df1.drop(index=0, inplace=True)

    df = pd.concat([df1.iloc[:-41], df2], axis=0)
    df.set_index(pd.to_datetime(df['Date']+' '+df['Heure'], format="%d/%m/%Y %H:%M:%S"), inplace=True)

    df.to_pickle('Data/df.pkl')

df = pd.read_pickle('Data/df.pkl')

# Plot 

In [ ]:
df['business_day'] = np.is_busday(df.index.round('1D').values.astype('datetime64[D]'))

feat = 'PT (1 min)'
df[f'{feat}_q1'] = df[feat].rolling(60).quantile(.25)
df[f'{feat}_q3'] = df[feat].rolling(60).quantile(.75)
df[f'{feat}_mean'] = df[feat].rolling(120).mean()

In [ ]:
profile = pd.DataFrame(index=df['Heure'].unique())

profile['PT_q1'] = df.loc[df['business_day'], [f'{feat}_q1', 'Heure']].groupby('Heure').median()
profile['PT_q3'] = df.loc[df['business_day'], [f'{feat}_q3', 'Heure']].groupby('Heure').median()
profile['PT_mean'] = df.loc[df['business_day'], [f'{feat}_mean', 'Heure']].groupby('Heure').median()

df.loc[df['business_day'], 'PT_profile_q1'] = df['Heure'].apply(lambda x: profile.loc[x, 'PT_q1'])
df.loc[df['business_day'], 'PT_profile_q3'] = df['Heure'].apply(lambda x: profile.loc[x, 'PT_q3'])
df.loc[df['business_day'], 'PT_profile_mean'] = df['Heure'].apply(lambda x: profile.loc[x, 'PT_mean'])

In [ ]:
profile_we = pd.DataFrame(index=df['Heure'].unique())

profile_we['PT_q1'] = df.loc[~df['business_day'], [f'{feat}_q1', 'Heure']].groupby('Heure').median()
profile_we['PT_q3'] = df.loc[~df['business_day'], [f'{feat}_q3', 'Heure']].groupby('Heure').median()
profile_we['PT_mean'] = df.loc[~df['business_day'], [f'{feat}_mean', 'Heure']].groupby('Heure').median()

df.loc[~df['business_day'], 'PT_profile_q1'] = df['Heure'].apply(lambda x: profile_we.loc[x, 'PT_q1'])
df.loc[~df['business_day'], 'PT_profile_q3'] = df['Heure'].apply(lambda x: profile_we.loc[x, 'PT_q3'])
df.loc[~df['business_day'], 'PT_profile_mean'] = df['Heure'].apply(lambda x: profile_we.loc[x, 'PT_mean'])

In [ ]:
df_plot = pd.DataFrame(df[['PT_profile_mean', 'business_day', 'PT (1 min)_mean']].resample('60T').mean())

In [ ]:
df_plot['office'] = df_plot[['PT (1 min)_mean', 'PT_profile_mean']].min(axis=1)
df_plot['min_smooth'] = df_plot[['PT (1 min)_mean', 'PT_profile_mean']].min(axis=1).rolling(30).quantile(.2)
df_plot['baseline'] = df_plot[['min_smooth', 'PT (1 min)_mean', 'PT_profile_mean']].min(axis=1)

In [ ]:
percentages = 100*df_plot[['baseline', 'office', 'PT (1 min)_mean']].div(df_plot[['baseline', 'office', 'PT (1 min)_mean']].sum(axis=1), axis=0)

In [ ]:
fig = go.Figure([
                    go.Scatter(x=df_plot.index, y=df_plot['baseline'], text=percentages['baseline'], name='Baseline', stackgroup='one', line_color='#636EFA', hovertemplate='%{text:.2f}%  (%{y:.2s}W)'),
                    go.Scatter(x=df_plot.index, y=df_plot['office']-df_plot['baseline'], text=percentages['office'], name='Offices', stackgroup='one', line_color='#FFA15A', hovertemplate='%{text:.2f}%  (%{y:.2s}W)'),
                    go.Scatter(x=df_plot.index, y=df_plot['PT (1 min)_mean']-df_plot['office'], text=percentages['PT (1 min)_mean'], name='Others', stackgroup='one', line_color='#00CC96', hovertemplate='%{text:.2f}%  (%{y:.2s}W)'),

                    go.Scatter(x=df_plot.index, y=df_plot['office'], name='Offices', line_color='#FFA15A', showlegend=False, hoverinfo='skip'),
                    go.Scatter(x=df_plot.index, y=df_plot['baseline'], name='Baseline', line_color='#636EFA', showlegend=False, hoverinfo='skip'),

                ])

fig.update_xaxes(showspikes=True, spikemode="across", spikedash='solid', spikesnap='cursor', title='Temps')
fig.update_yaxes(title='Puissance (W)')

fig.update_layout(hovermode="x unified", title='Répartition de la puissance')


fig.show()

In [ ]:
df.to_pickle('Data/df_profile.pkl')
df_plot.to_pickle('Data/df_plot.pkl')

# LSTM

In [ ]:
# Convert the dataframe to a numpy array
data = df[['P1 (1 min)', 'P2 (1 min)', 'P3 (1 min)', 'Q1 (1 min)', 'Q2 (1 min)', 'Q3 (1 min)']].to_numpy()

# Split the data into input and target sequences
timesteps = 30  # Length of the input sequence
X = []
y = []
for i in range(len(data) - timesteps):
    X.append(data[i:i+timesteps])
    y.append(data[i+timesteps])
X = np.array(X).astype(float)
y = np.array(y).astype(float)

# Standardize the data
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std

In [ ]:
# Load the timeseries data

# X.shape (41502, 10, 6)
N_sample = X.shape[0]

X_train, X_valid, X_test = X[:int(0.6*N_sample)], X[int(0.6*N_sample):int(0.8*N_sample)], X[int(0.8*N_sample):]

# Set the parameters for the autoencoder
latent_dim = 3  # Dimensionality of the latent space
n_feat = X.shape[2]

# Create the LSTM autoencoder model
model = Sequential()

# Encoder
model.add(LSTM(latent_dim, input_shape=(timesteps, n_feat), return_sequences=False, name='encode_LSTM'))
model.add(RepeatVector(timesteps, name='Repeat'))

# Decoder
model.add(LSTM(n_feat, return_sequences=True, name='decode_LTSM'))
model.add(TimeDistributed(Dense(n_feat), name='TD'))

model.summary()

In [ ]:
# Fit the model
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=False, validation_data=[X_valid, X_valid])

In [ ]:
encoder = Model(inputs=model.inputs, outputs=model.layers[0].output)

In [ ]:
components = encoder.predict(X_test)

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)


fig.add_trace(go.Scatter(x=df.index[-len(X_test):],y=components[:,0], name='component 1'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index[-len(X_test):],y=components[:,1], name='component 1'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index[-len(X_test):], y=df['PT (1 min)'], name='PT'), row=2, col=1)

fig.show()

In [ ]:
fig = go.Figure(go.Scatter(x=components[:,0], y=components[:,1], mode='markers'))
fig.show()